In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("dataset.csv")
# 切分数据集，将特征与标签分开
X = df.drop("target",axis = 1).values
Y = df["target"].values

In [ ]:
#划分训练集与测试集
X_train, X_test , Y_train , Y_test = train_test_split(X , Y , test_size = 0.2 , random_state =1)

#特征缩放
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 转化为tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).reshape(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).reshape(-1, 1)

In [ ]:
# 将数据切分批次
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset , batch_size = 32 , shuffle= True)

In [ ]:
#定义模型
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_size):
        super(HeartDiseaseModel, self).__init__()
        # 第一层
        self.layer1 = nn.Linear(input_size, 16)
        self.relu1 = nn.ReLU()

        # 第二层
        self.layer2 = nn.Linear(16,8)
        self.relu2 = nn.ReLU()

        # 输出层
        self.output_layer = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    # 前向传播
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.output_layer(x)
        x = self.sigmoid(x)
        return x
    

In [ ]:
# 初始化特征数量
input_size = X_train.shape[1]

# 初始化模型
model  = HeartDiseaseModel(input_size)

# 定义损失函数为二元交叉熵损失函数
Loss_function = nn.BCELoss()

# 定义优化器为随机梯度下降
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


In [ ]:
#设置训练轮数
epochs  = 10000

#开启训练模式
model.train()


for epoch in range(epochs):
    
    #初始化每轮损失
    epoch_loss = 0
    
    # 迭代每个批次
    for batch_X , batch_Y in train_loader :
        #初始化梯度，防止累加
        optimizer.zero_grad()
        #预测
        predictions = model(batch_X)
        #计算损失
        loss  = Loss_function(predictions , batch_Y)
        #反向传播
        loss.backward()
        #更新参数
        optimizer.step()
        #累计损失
        epoch_loss += loss.item()
    print(f"{epoch+1} , Loss : {epoch_loss/len(train_loader):.4f}")

In [ ]:
#开启评估模式
model.eval()
#设置不计算梯度，加快计算速度
with torch.no_grad():
    #对测试数据集进行预测
    test_predictions = model(X_test_tensor)
    #计算预测类别，概率大于等于0.5的归为有1类即有心脏病，否则为0类即无心脏病
    test_predictions_class = (test_predictions >= 0.5).float()
    #计算正确预测的数量
    correct_sums = (test_predictions_class == Y_test_tensor).sum().float()
    #计算正确率
    correct_rate = correct_sums / Y_test_tensor.shape[0]
    print(f"Test Correct Rate is: {correct_rate.item()*100:.2f}%")
